In [9]:
import csv
import cv2
import numpy as np

lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [33]:
images = []
measurements = []

# Correction factor
correction = 0.2

for line in lines:
    steering_center = float(line[3])

    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
#     print(current_path)
        image = cv2.imread(current_path)
        images.append(image)
#     print(measurement)
        if i == 0:
            measurements.append(steering_center)
        elif i == 1:
            measurements.append(steering_center + correction)  # streering_left
        elif i == 2:
            measurements.append(steering_center - correction) # steering_right
    

In [34]:
# Data Augmentation
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    # Flip the images
    augmented_images.append(cv2.flip(image, 1))
    # change the direction of the measurement
    augmented_measurements.append(measurement*-1.0)

In [35]:


X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)


In [36]:
X_train.shape

(14016, 160, 320, 3)

In [37]:
y_train.shape

(14016,)

In [38]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPool2D

In [39]:
model = Sequential()
# normalize the input data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Conv2D(6, (5, 5), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(6, (5, 5), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

model.save('model.h5')

Train on 11212 samples, validate on 2804 samples
Epoch 1/7
11212/11212 [==============================] - 347s - loss: 0.8215 - val_loss: 0.0472
Epoch 2/7
11212/11212 [==============================] - 336s - loss: 0.0154 - val_loss: 0.0353
Epoch 3/7
11212/11212 [==============================] - 350s - loss: 0.0114 - val_loss: 0.0307
Epoch 4/7
11212/11212 [==============================] - 369s - loss: 0.0088 - val_loss: 0.0306
Epoch 5/7
11212/11212 [==============================] - 337s - loss: 0.0069 - val_loss: 0.0330
Epoch 6/7
11212/11212 [==============================] - 343s - loss: 0.0058 - val_loss: 0.0344
Epoch 7/7
11212/11212 [==============================] - 362s - loss: 0.0050 - val_loss: 0.0357
